# Serialization, JSON, and Rich CLI Library

### Topics

- serializing objects
- deserializing objects
- pickle
- JSON
- Rich CLI Library

## Serializing objects

- objects created in memory are not persistent
    - they hold a lot of important information/states of objects as the program is being executed
- to make an object persistent, we need to create a series of bytes that represent the state of the object and write those bytes to a file
    - need to encode objects (**serializing**)
    - also need to decode objects from a series of bytes (**deserializing**)
- web services often use a service described as **RESTful**
    - REST - REpresentational State Transfer
- the server and client will exchange representation of objects using RESTful services
- several ways to serialize objects
- Pickle, JSON, XML, etc.

## Serializing objects using pickle

- Python `pickle` module is an object-oriented way to store object state directly in a special storage format
- essentially converts an object's state (and all the state of all the objects it holds as attributes) into a series of bytes
    - these bytes then can be transported or stored however we see fit
- pickled files are for temporary persistence
    - Pickled files using Python 3.7 may not be unpickled by newer version of Python or vice versa
- NOTE - pickle module is not secure
    - only unpickle data you trust!

In [2]:
import pickle

In [3]:
help(pickle)

Help on module pickle:

NAME
    pickle - Create portable serialized representations of Python objects.

MODULE REFERENCE
    https://docs.python.org/3.9/library/pickle
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    See module copyreg for a mechanism for registering custom picklers.
    See module pickletools source for extensive comments.
    
    Classes:
    
        Pickler
        Unpickler
    
    Functions:
    
        dump(object, file)
        dumps(object) -> string
        load(file) -> object
        loads(bytes) -> object
    
    Misc variables:
    
        __version__
        format_version
        compatible_formats

CLASSES
    builtins.Exception(builtins.BaseException)
      

In [4]:
some_data = ['a list', 'containing', 5, 'items', {"including": ("str", "int", "dict", "list", "tuple")}]

In [5]:
some_data

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')}]

In [6]:
# let's pickle/serialize the object list object
with open("pickled_list.pkl", 'wb') as file:
    pickle.dump(some_data, file)

In [7]:
# let's deserialize/load the object from file
with open("pickled_list.pkl", 'rb') as file:
    loaded_list = pickle.load(file)

In [8]:
type(loaded_list)

list

In [9]:
loaded_list

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')}]

In [10]:
some_data == loaded_list

True

In [11]:
# use loaded_list object
loaded_list.append([1, 2, 3])

In [12]:
loaded_list

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')},
 [1, 2, 3]]

## Serializing objects using JSON

- there are many text-based format to exchange data
- XML - Extensive Markup Language (XML)
- YAML - Yet Another Markup Language
- CSV - Comma-Separated Value
- most of these techniques have obscure features that can be exploited from a security point of view
    - e.g., may allow arbitrary commands to be executed on the host machine

### JSON - JavaScript Object Notation

- human-readable text-based format for exchanging data
- one of the most popular techniques used by RESTful API services
- JSON, as the name says, is more popular in JavaScript language to transfer data to a browser from the web server
- JSON though popular is not as robust as the **pickle** module
- it can serialize only basic data: 
    - integers, floats, strings, and simple containers such as lists and dictionaries
- generally, `json` module's functions try to serialize the object's state using the value of the object's `__dict__` attribute
- json provides **dump** and **dumps**, **load** and **loads** functions

In [13]:
import json

In [ ]:
help(json)

In [14]:
class Contact:
    def __init__(self, first, last):
        self.first = first
        self.last = last
        
    @property
    def full_name(self):
        return f'{self.first} {self.last}'

In [15]:
c = Contact("John", "Smith")

In [16]:
c.full_name

'John Smith'

In [17]:
json.dumps(c.__dict__)

'{"first": "John", "last": "Smith"}'

In [18]:
# better approach
from typing import Any
import json

class ContactEncoder(json.JSONEncoder):
    def default(self, obj: Any) -> Any:
        if isinstance(obj, Contact):
            return {
                "__class__": "Contact",
                "first": obj.first,
                "last": obj.last,
                "full_name": obj.full_name,
            }
        return super().default(obj)

In [19]:
c = Contact("John", "Smith")
text = json.dumps(c, cls=ContactEncoder)

In [20]:
text

'{"__class__": "Contact", "first": "John", "last": "Smith", "full_name": "John Smith"}'

In [21]:
def decode_contact(json_object: Any) -> Any:
    if json_object.get("__class__") == "Contact":
        return Contact(json_object["first"], json_object["last"])
    else:
        return json_object

In [22]:
c2 = json.loads(text, object_hook=decode_contact)

In [23]:
c2.full_name

'John Smith'

In [24]:
type(c2)

__main__.Contact

In [25]:
some_text = ('{"__class__": "Contact", "first": "Milli", "last": "Dale", '
            '"full_name": "Milli Dale"}')

In [26]:
c3 = json.loads(some_text, object_hook=decode_contact)

In [27]:
c3.full_name

'Milli Dale'

In [28]:
help(c3)

Help on Contact in module __main__ object:

class Contact(builtins.object)
 |  Contact(first, last)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, first, last)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  full_name
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



## JSON API and Weather App

- Weather data sources: Open Weather, Weather Stack, Google Search, etc.
- most services provide free API access with some limitations
- replace access_key/api_id with your API key in the following demo

### Weather Stack

- real-time & Historical World Weather Data API
- retrieve instant, accurate weather information for any location in the world in lightweight JSON format
- Visit [https://weatherstack.com/](https://weatherstack.com/) to create your free account and API keys

In [29]:
import json

In [30]:
! pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# get current weather info for Grand Junction, Colorado
# using weatherstack.com API

import requests

params = {
    'access_key': '',  # FIXME
  'query': 'Grand Junction, Colorado'
}

api_result = requests.get('http://api.weatherstack.com/current', params) 
# only allows http access with free account

api_response = api_result.json()

/Users/rbasnet/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [32]:
api_response

{'request': {'type': 'City',
  'query': 'Grand Junction, United States of America',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Grand Junction',
  'country': 'United States of America',
  'region': 'Colorado',
  'lat': '39.064',
  'lon': '-108.550',
  'timezone_id': 'America/Denver',
  'localtime': '2025-10-16 12:10',
  'localtime_epoch': 1760616600,
  'utc_offset': '-6.0'},
 'current': {'observation_time': '06:10 PM',
  'temperature': 11,
  'weather_code': 113,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'],
  'weather_descriptions': ['Sunny'],
  'astro': {'sunrise': '07:25 AM',
   'sunset': '06:33 PM',
   'moonrise': '02:35 AM',
   'moonset': '04:29 PM',
   'moon_phase': 'Waning Crescent',
   'moon_illumination': 27},
  'air_quality': {'co': '132.85',
   'no2': '2.75',
   'o3': '104',
   'so2': '1.35',
   'pm2_5': '5.55',
   'pm10': '36.85',
   'us-epa-index': '1',
   'gb-defra-index': '1'},
  'wind_speed': 4,


In [33]:
# print better
print(json.dumps(api_response, indent=2))

{
  "request": {
    "type": "City",
    "query": "Grand Junction, United States of America",
    "language": "en",
    "unit": "m"
  },
  "location": {
    "name": "Grand Junction",
    "country": "United States of America",
    "region": "Colorado",
    "lat": "39.064",
    "lon": "-108.550",
    "timezone_id": "America/Denver",
    "localtime": "2025-10-16 12:10",
    "localtime_epoch": 1760616600,
    "utc_offset": "-6.0"
  },
  "current": {
    "observation_time": "06:10 PM",
    "temperature": 11,
    "weather_code": 113,
    "weather_icons": [
      "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png"
    ],
    "weather_descriptions": [
      "Sunny"
    ],
    "astro": {
      "sunrise": "07:25 AM",
      "sunset": "06:33 PM",
      "moonrise": "02:35 AM",
      "moonset": "04:29 PM",
      "moon_phase": "Waning Crescent",
      "moon_illumination": 27
    },
    "air_quality": {
      "co": "132.85",
      "no2": "2.75",
      "o3": "104",
    

In [34]:
def celciusToFahrenheit(celcius: int) -> float:
    return celcius*(9/5)+32 

In [35]:
celciusTemp = api_response['current']['temperature']

In [36]:
print('Current temperature in %s is %d℃' % (api_response['location']['name'], celciusTemp))

Current temperature in Grand Junction is 11℃


In [37]:
print(f"Current temperature in {api_response['location']['name']} is \
    {celciusToFahrenheit(celciusTemp)} {chr(8457)}")

Current temperature in Grand Junction is     51.8 ℉


### OpenWeather

- Weather forecasts, nowcasts and history in a fast and elegant way
- visit [https://openweathermap.org/](https://openweathermap.org/) to create a free account and create API keys
- it takes a couple of hours to activate your API
- 1,000 API calls per day for free
- 0.0015 USD per API call over the daily limit

In [ ]:
# get current weather info for Grand Junction, Colorado
# using openweathermap.org API
import requests

params = {
    'appid': '',  # FIXME
  'q': 'Grand Junction, Colorado'
}

url = 'https://api.openweathermap.org/data/2.5/weather'

# https is supported for free tier
api_result = requests.get(url, params)

api_response = api_result.json()

In [39]:
api_response

{'coord': {'lon': -108.5507, 'lat': 39.0639},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 285.87,
  'feels_like': 284.32,
  'temp_min': 284.95,
  'temp_max': 286.67,
  'pressure': 1015,
  'humidity': 43,
  'sea_level': 1015,
  'grnd_level': 832},
 'visibility': 10000,
 'wind': {'speed': 5.66, 'deg': 330},
 'clouds': {'all': 0},
 'dt': 1760637966,
 'sys': {'type': 2,
  'id': 2020637,
  'country': 'US',
  'sunrise': 1760621093,
  'sunset': 1760661248},
 'timezone': -21600,
 'id': 5423573,
 'name': 'Grand Junction',
 'cod': 200}

In [40]:
import json
print(json.dumps(api_response, indent=2))

{
  "coord": {
    "lon": -108.5507,
    "lat": 39.0639
  },
  "weather": [
    {
      "id": 800,
      "main": "Clear",
      "description": "clear sky",
      "icon": "01d"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 285.87,
    "feels_like": 284.32,
    "temp_min": 284.95,
    "temp_max": 286.67,
    "pressure": 1015,
    "humidity": 43,
    "sea_level": 1015,
    "grnd_level": 832
  },
  "visibility": 10000,
  "wind": {
    "speed": 5.66,
    "deg": 330
  },
  "clouds": {
    "all": 0
  },
  "dt": 1760637966,
  "sys": {
    "type": 2,
    "id": 2020637,
    "country": "US",
    "sunrise": 1760621093,
    "sunset": 1760661248
  },
  "timezone": -21600,
  "id": 5423573,
  "name": "Grand Junction",
  "cod": 200
}


In [41]:
max_temp = api_response['main']['temp_max'] # temperature is in Kelvin

In [42]:
max_temp 

286.67

In [43]:
curr_temp = api_response['main']['temp']

In [44]:
curr_temp

285.87

In [45]:
print(f'Current Temperature in {api_response["name"]} is: {celciusToFahrenheit(curr_temp-273.15):.1f}{chr(8457)}')

Current Temperature in Grand Junction is: 54.9℉


In [46]:
api_response['main']['temp_min']

284.95

## Using Google Search

- When you search for places in Google, it provides some weather info as well
- use `beautifulsoup4` python package to parse html data
- muse pip install it; already installed in the docker container

In [ ]:
from bs4 import BeautifulSoup
import requests

In [48]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def getGoogleWeather(city):
    city = city.replace(" ", "+")
    url = f'https://www.google.com/search?q={city}'
    res = requests.get(url, headers=headers)
    print("Searching...\n")
    print(f'{res=}')
    print(res.text)
    soup = BeautifulSoup(res.text, 'html.parser')
    location_tag = soup.find_all('div')
    print(f'{len(location_tag)}')
    location = soup.select_one('.BBwThe').getText().strip() # search by class
    time = soup.select('#wob_dts')[0].getText().strip() #search by id
    forecast = soup.select('#wob_dc')[0].getText().strip()
    temp = soup.select('#wob_tm')[0].getText().strip()
    return location, time, forecast, temp


In [49]:
city = input("Enter the Name of a City ->  ")
city = city+" weather"
result = getGoogleWeather(city)

Searching...

res=<Response [200]>
<!DOCTYPE html><html lang="en"><head><title>Google Search</title><style>body{background-color:#fff}</style><script nonce="YGJ95VOdbsybnZ1WkUNFOQ">window.google = window.google || {};window.google.c = window.google.c || {cap:0};</script></head><body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=tTXxaPvNG_6Z0PEP4ruj8Ag" http-equiv="refresh"><div style="display:block">Please click <a href="/httpservice/retry/enablejs?sei=tTXxaPvNG_6Z0PEP4ruj8Ag">here</a> if you are not redirected within a few seconds.</div></noscript><script nonce="YGJ95VOdbsybnZ1WkUNFOQ">(function(){var sctm=false;(function(){sctm&&google.tick("load","pbsst");}).call(this);})();</script><script nonce="YGJ95VOdbsybnZ1WkUNFOQ">//# sourceMappingURL=data:application/json;charset=utf-8;base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOlsiICJdLCJuYW1lcyI6WyJjbG9zdXJlRHluYW1pY0J1dHRvbiJdLCJtYXBwaW5ncyI6IkFBQUE7QUFB

NameError: name 'BeautifulSoup' is not defined

In [ ]:
result

### Weather Forecast

- openweathermap.org provides 5-day 3-hour forecast data for freemium account

In [ ]:
# get current weather using openweathermap.org API
import requests

def forecastWeather(city) -> json:
    params = {
        'appid': '',  # FIXME
      'q': city,
      'units': 'imperial' # uses US metrics
    }

    #url = httpsapi.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
    url = 'https://api.openweathermap.org/data/2.5/forecast'

    api_result = requests.get(url, params)

    api_response = api_result.json()
    return api_response

In [51]:
city = input('Enter name of a City: ')

In [52]:
json_response = forecastWeather(city)

In [53]:
json_response

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1760648400,
   'main': {'temp': 65.12,
    'feels_like': 63.27,
    'temp_min': 65.03,
    'temp_max': 65.12,
    'pressure': 1002,
    'sea_level': 1002,
    'grnd_level': 828,
    'humidity': 41,
    'temp_kf': 0.05},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 11.07, 'deg': 240, 'gust': 19.55},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2025-10-16 21:00:00'},
  {'dt': 1760659200,
   'main': {'temp': 59.32,
    'feels_like': 56.14,
    'temp_min': 56.39,
    'temp_max': 59.32,
    'pressure': 1002,
    'sea_level': 1002,
    'grnd_level': 828,
    'humidity': 25,
    'temp_kf': 1.63},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 1},
   'wind': {'speed': 3.31, 'deg': 22, 'gust': 4.5},
   'visibility': 10000,
   '

In [ ]:
from datetime import datetime

print('Forecast Result for: ', json_response['city']['name'])
for forecast in json_response['list']:
    print('Day: ', datetime.fromtimestamp(forecast['dt']))
    feelsLike = f"{forecast['main']['feels_like']} {chr(8457)}"
    temp = f"{forecast['main']['temp']} {chr(8457)}"
    print(f"Feels like: {feelsLike} Temperature: {temp}")

# Rich CLI Library

- Textualize provides various libraries to make UI better - [https://www.textualize.io/](https://www.textualize.io/)
- Rich is a Python library for rich text and beautiful formatting in the terminal
- Rich can be used to make your Python CLI apps look better
- [https://github.com/Textualize/rich](https://github.com/Textualize/rich)
- See an application of Rich: [https://github.com/rambasnet/kattis-cli](https://github.com/rambasnet/kattis-cli)
- Kattis-cli uses trogon to display TUI - https://github.com/Textualize/trogon

## Exercise

1. Find average temperate for Grand Junction in the 1st week of January 2000.
2. Create a simple weather app using any API of your choice.
3. Use Rich library to make your CLI app look better.